In [1]:
#perform imports and load dataset
import numpy as np 
import pandas as pd
import spacy
import re # For regular expressions|||
import string # For handling string
import nltk
import pickle
import sklearn
import matplotlib.pyplot as plt
from nltk.util import ngrams
import seaborn as sns
from nltk.corpus import stopwords
from collections import  Counter
import gensim
nltk.download('punkt')
nltk.download('wordnet')
from matplotlib import pyplot
from numpy import array 
from textblob import TextBlob

[nltk_data] Downloading package punkt to /Users/lise-
[nltk_data]     mariehugo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lise-
[nltk_data]     mariehugo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
df1 = pd.read_excel(r'/Users/lise-mariehugo/Desktop/Thesisdata/FinDataLabelled/2021/X2021_2.xlsx') 

In [17]:
#drop duplicates
df1 = df1.drop_duplicates(keep='first')


#removing numbers
df1['text'] = df1['text'].astype('string')
df1.dropna(subset=['text'], inplace=True)

In [18]:
#lowercase
df1['text']=df1['text'].apply(lambda x: x.lower())

#print(textDF.head())
#itworked!

#removing mentions
df1['text']=df1['text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+",'', x))

#removing numbers and words containing numbers
df1['text']=df1['text'].apply(lambda x: re.sub('\w*\d\w*','', x))

#removing hashtags?
#df1['text']=df1['text'].apply(lambda x: re.sub("#[A-Za-z0-9_]+",'', x))
#df1['text']=df1['text'].apply(lambda x: re.split('#|_', x))
df1['text']=df1['text'].apply(lambda x: " ".join(word.strip() for word in re.split('#|_', x)))


#removing links
df1['text']=df1['text'].apply(lambda x: re.sub(r"http\S+",'', x))
df1['text']=df1['text'].apply(lambda x: re.sub(r"www.\S+",'', x))


#removing punctuation
df1['text']=df1['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

#removing empty spaces and special characters
df1['text']=df1['text'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))

df1.shape
df1.head()

,index,author_id,edit_history_tweet_ids,id,possibly_sensitive,created_at,attachments.media_keys,attachments.poll_ids,text,label,...,urls,mentions,cashtags,retweet_count,reply_count,like_count,quote_count,place_id,coordinates.type,coordinates.coordinates
0,1,43572706,1.36613477603443E+18,1.36613477603443E+18,FALSE,2021-02-28T21:14:46.000Z,3_1366134772276350989,NaN,neil ellis sauvignon blanc seductively ripe go...,NaN,...,"list(start = 219, end = 242, url = ""https://t....",NaN,NaN,0.0,0.0,2.0,0.0,07d9ed873f082002,NaN,NaN
1,2,43572706,1.3661332253582899E+18,1.3661332253582899E+18,FALSE,2021-02-28T21:08:37.000Z,3_1366133220346060803,NaN,beautiful zebra roaming freely in groenekloof ...,NaN,...,"list(start = 226, end = 249, url = ""https://t....",NaN,NaN,0.0,0.0,0.0,0.0,07d9e4cdad486002,NaN,NaN
2,3,43572706,1.3661315495409101E+18,1.3661315495409101E+18,FALSE,2021-02-28T21:01:57.000Z,3_1366131545287852035,NaN,potato amp brinjal curry fried southafrica mig...,NaN,...,"list(start = 196, end = 219, url = ""https://t....",NaN,NaN,0.0,0.0,0.0,0.0,e564d30dc173d2a8,NaN,NaN
3,4,625546488,1.36612954543951E+18,1.36612954543951E+18,FALSE,2021-02-28T20:53:59.000Z,NaN,NaN,what did the president say guys hape rona ba b...,NaN,...,NaN,NaN,NaN,0.0,1.0,1.0,0.0,dfc037f64a373ac8,NaN,NaN
4,5,7.8937670123512998E+17,1.36612730345823E+18,1.36612730345823E+18,FALSE,2021-02-28T20:45:05.000Z,NaN,NaN,capitec elect token receipt no electricity vat...,NaN,...,NaN,"list(start = 134, end = 143, username = ""Eskom...",NaN,0.0,0.0,0.0,0.0,dd9c0d7d7e07eb49,NaN,NaN


In [19]:
#stopwords
import spacy
nlp = spacy.load('en_core_web_sm')
# Print the set of spaCy's default stop words (remember that sets are unordered):
print(nlp.Defaults.stop_words)
#add twitter as a stop word and rt
# Add the word to the set of stop words. Use LOWERCASE!
nlp.Defaults.stop_words.add('twitter')
nlp.Defaults.stop_words.add('rt')
nlp.Defaults.stop_words.add('s')
nlp.Defaults.stop_words.add('tweet')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('follower')
nlp.Defaults.stop_words.add('null')
nlp.Defaults.stop_words.add('http')
nlp.Defaults.stop_words.add('url')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('trending')

# Set the stop_word tag on the lexeme
nlp.vocab['twitter'].is_stop = True
nlp.vocab['rt'].is_stop = True
nlp.vocab['s'].is_stop = True
nlp.vocab['tweet'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['follower'].is_stop = True
nlp.vocab['null'].is_stop = True
nlp.vocab['http'].is_stop = True
nlp.vocab['url'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['trending'].is_stop = True

stop = nlp.Defaults.stop_words

{'’ll', 'here', 'nor', '’s', 'out', 'alone', 'seeming', 'eight', 'except', 'top', 'from', 'hundred', 'indeed', "'m", 'does', 'through', 'whose', 'yours', 'twenty', 'serious', 'wherever', 'sixty', 'wherein', 'on', 'once', 'put', 'sometimes', 'mostly', 'was', 'trend', 'regarding', 'more', 'therein', '‘d', 'again', 'could', 'its', 'nine', 'empty', 'every', 'do', 'anyway', 'take', 'mine', 'anything', 'already', 'under', 'each', 'behind', 'ca', 'give', 'doing', 'perhaps', 'because', 'ten', 'less', 'them', 'twelve', 'two', 'as', 'toward', '‘re', 'quite', 'besides', 'there', 'other', 'thru', 'who', 'none', 'become', 'is', 'unless', 'via', 'rather', 'we', 'when', 'amount', 'below', 'hereupon', 'thereafter', 'my', 'http', 'where', 'since', 'make', 'up', 'beforehand', 'therefore', 'someone', 'first', 'into', 'still', 'around', 'not', 'five', 'amongst', 'done', 'beyond', 'her', 'of', '’ve', 'cannot', 'noone', '‘ll', 'down', 'seemed', 'whereas', '‘ve', 'everything', 'twitter', 'may', 'everyone', '

In [20]:
#lemmatisation
# Loading model
nlp = spacy.load('en_core_web_sm')

# Lemmatization with stopwords removal
#df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False)&((token.pos_=="ADJ")|(token.pos_=="NOUN")|(token.pos_=="VERB")))]))
#df1.head()

df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False))]))

In [21]:
#getting rid of empty text rows
df1.dropna(subset = "text", inplace=True)
df1.shape

(4467, 26)

In [22]:
#VADER SENTIMENT
#sentiment column
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()
#pip install vaderSentiment

#add a new column 
lists = list(range(len(df1)))
df1["Sentiment"] = lists

#aply function to get sentiment for each row
#df1['Sentiment'] = df1['text'].apply(lambda x: sid_obj.polarity_scores(x)['compound'])
result = []

df1["text"] = pd.DataFrame(df1["text"] )
type(df1["text"] )

for index, row in df1.iterrows():
     text = row['text']
     sentiment  = sid_obj.polarity_scores(text)['compound']
     result.append((sentiment))

df1['VaderSentiment'] = result


In [23]:
df1.head()

,index,author_id,edit_history_tweet_ids,id,possibly_sensitive,created_at,attachments.media_keys,attachments.poll_ids,text,label,...,cashtags,retweet_count,reply_count,like_count,quote_count,place_id,coordinates.type,coordinates.coordinates,Sentiment,VaderSentiment
0,1,43572706,1.36613477603443E+18,1.36613477603443E+18,FALSE,2021-02-28T21:14:46.000Z,3_1366134772276350989,NaN,neil ellis sauvignon blanc seductively ripe go...,NaN,...,NaN,0.0,0.0,2.0,0.0,07d9ed873f082002,NaN,NaN,0,0.0000
1,2,43572706,1.3661332253582899E+18,1.3661332253582899E+18,FALSE,2021-02-28T21:08:37.000Z,3_1366133220346060803,NaN,beautiful zebra roam freely groenekloof nature...,NaN,...,NaN,0.0,0.0,0.0,0.0,07d9e4cdad486002,NaN,NaN,1,0.7783
2,3,43572706,1.3661315495409101E+18,1.3661315495409101E+18,FALSE,2021-02-28T21:01:57.000Z,3_1366131545287852035,NaN,potato amp brinjal curry fry southafrica migue...,NaN,...,NaN,0.0,0.0,0.0,0.0,e564d30dc173d2a8,NaN,NaN,2,0.0000
3,4,625546488,1.36612954543951E+18,1.36612954543951E+18,FALSE,2021-02-28T20:53:59.000Z,NaN,NaN,president guy hape rona ba bangwe knock ramaphosa,NaN,...,NaN,0.0,1.0,1.0,0.0,dfc037f64a373ac8,NaN,NaN,3,0.0000
4,5,7.8937670123512998E+17,1.36612730345823E+18,1.36612730345823E+18,FALSE,2021-02-28T20:45:05.000Z,NaN,NaN,capitec elect token receipt electricity vat un...,NaN,...,NaN,0.0,0.0,0.0,0.0,dd9c0d7d7e07eb49,NaN,NaN,4,0.0000


In [24]:
#TEXTBLOB

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

 #Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
  


In [25]:
result = []

for index, row in df1.iterrows():
     text = row['text']
     sentiment  = TextBlob(text).sentiment.polarity
     result.append((sentiment))

df1['TBSentiment'] = result

In [26]:
df1.head()

,index,author_id,edit_history_tweet_ids,id,possibly_sensitive,created_at,attachments.media_keys,attachments.poll_ids,text,label,...,retweet_count,reply_count,like_count,quote_count,place_id,coordinates.type,coordinates.coordinates,Sentiment,VaderSentiment,TBSentiment
0,1,43572706,1.36613477603443E+18,1.36613477603443E+18,FALSE,2021-02-28T21:14:46.000Z,3_1366134772276350989,NaN,neil ellis sauvignon blanc seductively ripe go...,NaN,...,0.0,0.0,2.0,0.0,07d9ed873f082002,NaN,NaN,0,0.0000,0.000
1,2,43572706,1.3661332253582899E+18,1.3661332253582899E+18,FALSE,2021-02-28T21:08:37.000Z,3_1366133220346060803,NaN,beautiful zebra roam freely groenekloof nature...,NaN,...,0.0,0.0,0.0,0.0,07d9e4cdad486002,NaN,NaN,1,0.7783,0.625
2,3,43572706,1.3661315495409101E+18,1.3661315495409101E+18,FALSE,2021-02-28T21:01:57.000Z,3_1366131545287852035,NaN,potato amp brinjal curry fry southafrica migue...,NaN,...,0.0,0.0,0.0,0.0,e564d30dc173d2a8,NaN,NaN,2,0.0000,0.000
3,4,625546488,1.36612954543951E+18,1.36612954543951E+18,FALSE,2021-02-28T20:53:59.000Z,NaN,NaN,president guy hape rona ba bangwe knock ramaphosa,NaN,...,0.0,1.0,1.0,0.0,dfc037f64a373ac8,NaN,NaN,3,0.0000,0.000
4,5,7.8937670123512998E+17,1.36612730345823E+18,1.36612730345823E+18,FALSE,2021-02-28T20:45:05.000Z,NaN,NaN,capitec elect token receipt electricity vat un...,NaN,...,0.0,0.0,0.0,0.0,dd9c0d7d7e07eb49,NaN,NaN,4,0.0000,0.800


In [27]:
#TD-IDF extraction
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer

df1.shape

#min_df=2
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df1["text"])

Y = X.toarray()
    

#feat_dict=vectorizer.vocabulary_.keys()
nummat = pd.DataFrame(data=Y, columns=vectorizer.get_feature_names_out())

counter = nummat.sum()


#TD-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
#print(corpus2)
X_train_tfidf = tfidf_transformer.fit_transform(X)
Yn = X_train_tfidf.toarray()
#checking type
#test.dtypes()
nummat2 = pd.DataFrame(data=Y, columns=vectorizer.get_feature_names_out())

nummat2.shape



(4467, 11682)

In [28]:
df1 = df1.reset_index(drop=True)
finaldf = pd.concat([df1, nummat2], axis=1)
del finaldf["Sentiment"] 
del finaldf["VaderSentiment"] 
del finaldf["TBSentiment"] 
del finaldf["source"] 
del finaldf["conversation_id"] 
#del finaldf["possibly_sensitive"] 
del finaldf["lang"] 
del finaldf["edit_history_tweet_ids"] 
del finaldf["id"] 
del finaldf["author_id"] 
#del finaldf["attachments"]
del finaldf["referenced_tweets"] 
del finaldf["in_reply_to_user_id"] 
#del finaldf["urls"] 
#del finaldf["mentions"] 
#del finaldf["annotations"] 
del finaldf["retweet_count"] 
del finaldf["reply_count"] 
del finaldf["like_count"] 
del finaldf["quote_count"]
del finaldf["place_id"] 
del finaldf["coordinates.type"] 
del finaldf["coordinates.coordinates"] 
#del finaldf["label"]
#del finaldf["hashtags"] 
finaldf.to_csv(r'/Users/lise-mariehugo/Desktop/Thesisdata/DataSetsForEachYear/2021/FEB21/TDIDF_FEB21.csv')

In [29]:
#sentiment data
sentdat = df1[["text", "VaderSentiment", "TBSentiment", "created_at"]]
sentdat.to_csv(r'/Users/lise-mariehugo/Desktop/Thesisdata/DataSetsForEachYear/2021/FEB21/SENT_FEB21.csv')